In [1]:
from ab_trainer import *
#from ab_classifier_free_guidance import *
from ab_denoising_diffusion_pytorch import *
from user_hints import *
import matplotlib.pyplot as plt
import torch
from multiprocessing import cpu_count
import os
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names,concatenate_datasets,DatasetDict,load_from_disk
from datasets import Dataset as HFDataset
from torchvision.datasets import ImageFolder
from torchviz import make_dot
from torchsummary import summary

In [2]:
#loa dimagenet
image_dir_val = "./data/sem images/val"
image_dir_train = "./data/sem images/train"

dataset_train = load_dataset("imagefolder", data_dir=image_dir_train)["train"]
dataset_val = load_dataset("imagefolder", data_dir=image_dir_val)["train"]



Resolving data files:   0%|          | 0/1098 [00:00<?, ?it/s]

Using custom data configuration default-999bcfdae24507ee
Reusing dataset imagefolder (/Home/siv32/eve036/.cache/huggingface/datasets/imagefolder/default-999bcfdae24507ee/0.0.0/0fc50c79b681877cc46b23245a6ef5333d036f48db40d53765a68034bc48faff)


  0%|          | 0/1 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/100 [00:00<?, ?it/s]

Using custom data configuration default-ccb5beaacb3b8920
Reusing dataset imagefolder (/Home/siv32/eve036/.cache/huggingface/datasets/imagefolder/default-ccb5beaacb3b8920/0.0.0/0fc50c79b681877cc46b23245a6ef5333d036f48db40d53765a68034bc48faff)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
train_folder = "data/tinyimgnet/train"
test_folder = "data/tinyimgnet/val"

In [4]:
device = torch.device(7 if torch.cuda.is_available() else "cpu")
print("Selected GPU:", torch.cuda.get_device_name(device))
#check wicj gpu is selected
torch.cuda.set_device(device)

print("Number of available GPUs:", torch.cuda.device_count())
print("device:", device)
print("Selected GPU:", torch.cuda.current_device())


Selected GPU: A100-SXM4-80GB
Number of available GPUs: 8
device: cuda:7
Selected GPU: 7


In [7]:

results_folder = "./results"

def load(milestone):
    print(str(results_folder  + f'/model-{milestone}.pt'))
    data = torch.load(str(results_folder  + f'/model-{milestone}.pt'), map_location=device)
    return data



In [8]:
model = ABUnet(
    dim = 64,
    out_dim = 2,
    channels=5,    

).to(device)
diffusion = ABGaussianDiffusion(
    model,
        image_size = 256,
        timesteps = 1000,
        objective = 'pred_v',
        beta_schedule = 'cosine',
        auto_normalize=False
).to(device)
loaded_data = load(88)
diffusion.load_state_dict(loaded_data['model'])

sampling timesteps 1000
timesteps 1000
./results/model-88.pt


<All keys matched successfully>

In [9]:
hint_generator = RandomHintGenerator(
    input_size = 256,
    hint_size = 2,
    num_hint_range = [0, 10]
    
)

In [10]:
trainer = ABTrainer(
        diffusion_model = diffusion,
        folder_train = train_folder,
        folder_sample= test_folder,
        train_ds = dataset_train,
        sample_ds= dataset_val,
        train_batch_size = 32,
        gradient_accumulate_every = 1,
        augment_horizontal_flip = False,
        train_lr = 5e-7,
        train_num_steps = 1000,
        ema_update_every = 10,
        ema_decay = 0.995,
        adam_betas = (0.9, 0.99),
        num_samples = 16,
        results_folder = './results',
        amp = False,
        mixed_precision_type = 'fp16',
        split_batches = True,
        calculate_fid = False,
        inception_block_idx = 2048,
        max_grad_norm = 1.,
        num_fid_samples = 50000,
        save_best_and_latest_only = False,
        #ab diffusion stuff
        hint_generator = hint_generator,
        hint_color_avg = True,
        log_freq = 100,
        save_every = 999,
        
        

        )

32
32
loading datasets:
time to load train dataset:  0.0001704692840576172
train dataset length:  1098
time to load sample dataset:  6.246566772460938e-05
sample dataset length:  100


In [11]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

/Home/siv32/eve036/.local/lib/python3.6/site-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


sampling loop time step:   0%|          | 0/1000 [00:00<?, ?it/s]

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
